In [ ]:
! pip install transformers accelerate torch torchvision pillow


In [ ]:
! python.exe -m pip install --upgrade pip

In [ ]:
! pip install ipywidgets

In [ ]:
! pip install requests

In [ ]:
! pip install ollama

In [3]:
import base64
from PIL import Image, ImageEnhance
from io import BytesIO
import ollama
import json
import re

def enhance_image(image_path):
    """Preprocess image for better text recognition"""
    img = Image.open(image_path)
    
    # Convert to grayscale
    if img.mode != 'L':
        img = img.convert('L')
    
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2.0)  # Increase contrast
    
    # Sharpen image
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(1.5)
    
    return img

def img_to_b64(image_path):
    """Convert enhanced image to base64"""
    img = enhance_image(image_path)
    buf = BytesIO()
    img.save(buf, format="PNG", quality=100)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

def clean_qwen_output(text):
    """Extract and validate JSON from Qwen response"""
    text = text.replace('```json', '').replace('```', '').strip()
    
    try:
        data = json.loads(text)
        if not isinstance(data, dict):
            raise ValueError("Output is not a JSON object")
        return data
    except json.JSONDecodeError:
        # Fallback: extract potential JSON
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group())
            except json.JSONDecodeError:
                pass
        return {"error": "Could not parse JSON output", "raw_response": text}

def extract_text_with_qwen(image_path):
    """Enhanced text extraction with Qwen2.5-VL"""
    client = ollama.Client()
    
    # System prompt for better structured output
    system_prompt = """You are an expert OCR system.
    Extract all visible text exactly as written from the document image, including handwritten and printed text.
    Return ONLY a valid JSON object with:
    - raw_text: Complete extracted text
    - lines: List of text lines
    - fields: Key-value pairs of identified fields (if any)"""
    
    try:
        img_b64 = img_to_b64(image_path)
        
        response = client.chat(
            model="qwen2.5vl:7b",
            messages=[
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": "Extract all text from this document exactly as written, including handwritten and printed text.",
                    "images": [img_b64]
                }
            ],
            options={
                "temperature": 0.1,  # Deterministic output
                "num_ctx": 4096      # Increase context window
            }
        )
        
        result = clean_qwen_output(response['message']['content'])
        result["success"] = True if "raw_text" in result else False
        return result
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "raw_response": None
        }

# Example usage
if __name__ == "__main__":
    result = extract_text_with_qwen("0388 1.jpg")
    print(json.dumps(result, indent=2, ensure_ascii=False))


{
  "raw_text": "Baltimore City Health Department\n50 0695\nS-356 50-01538\nCERTIFICATE OF DEATH\nBIRTH NO.\n50 0695\n1. NAME OF DECEASED\nBobby Boy Stiner\n2. DATE OF DEATH\nJanuary 24 1950\n3. PLACE OF DEATH\nBaltimore, Maryland\n4. USUAL RESIDENCE (Where deceased lived. If institution or residence)\nJOHNS HOPKINS HOSPITAL\n5. SEX\nMale\n6. COLOR OR RACE\nWhite\n7. WIDOWED, MARRIED, DIVORCED (Specify)\nChild\n8. DATE OF BIRTH\n1-23-50\n9. AGE (In years last birthday)\n2\n10. USUAL OCCUPATION (Give kind of work done during most of working life, even if retired)\n\n10A. KIND OF BUSINESS OR INDUSTRY\n\n11. BIRTHPLACE (State or foreign country)\n\n12. CITIZEN OF WHAT COUNTRY\n\n13. FATHER'S NAME\nRobert Stiner\n14. MOTHER'S MAIDEN NAME\n\n15. WAS DECEASED EVER IN U.S. ARMED FORCES? (Yes, no or unknown)\n\n16. SOCIAL SECURITY NO.\n\n17. INFORMANT\nJOHNS HOPKINS HOSPITAL\n\n18. CAUSE OF DEATH\nCONGENITAL ATELECTARIS\nDUE TO\nPREMATURITY\nDUE TO\nCERIORAL REDION\n\n19. DATE OF OPERATION\n\n